In [1]:
import tweepy
import json

api_key = "aJTgi4d1H1zmQNkQuHeualNhP"
api_secrets = "yPFdp2Bbib25r1iPQCqBuPq8B9UzoJbIdpv1jgzEZFEg1eJl6X"
access_token = "1409382627124019204-oPpzVGuCwyFfQTfoocFhgHi68whhog"
access_secret = "esgwpH5gnRfNB0SpiHkO52mZSI5VKKnn8SG2pafEnzj2b"
 
# Authenticate to Twitter
auth = tweepy.OAuthHandler(api_key,api_secrets)
auth.set_access_token(access_token,access_secret)
 
api = tweepy.API(auth)

TWEET_OBJECT_PATH = 'tweet-objects/'

 
try:
    api.verify_credentials()
    print('Successful Authentication')
except:
    print('Failed Authentication')

Successful Authentication


In [2]:
def get_ids(filename):
    with open(filename) as f:
        tweet_ids = f.readlines()

    source_ids = []
    reply_ids_dict = {}
    for i in range(len(tweet_ids)):
        all_ids = tweet_ids[i].split(",")
        for i in range(len(all_ids)):
            all_ids[i] = all_ids[i].replace("\n","")
        source_id = all_ids[0]
        source_ids.append(source_id)
        if len(all_ids) > 1:
            source_id = source_id.replace("\n","")
            reply_ids_dict[source_id] = all_ids[1:]

    return source_ids, reply_ids_dict

def get_labels(filename):    
    with open(filename) as f:
        tweet_labels = f.readlines()

    source_labels = []
    for i in range(len(tweet_labels)):
        label = tweet_labels[i]
        label = label.replace("\n","")
        source_labels.append(label)

    return source_labels

In [3]:
train_source_ids, train_reply_ids = get_ids('train.data.txt')
train_source_labels = get_labels('train.label.txt')

dev_source_ids, dev_reply_ids = get_ids('dev.data.txt')
dev_source_labels = get_labels('dev.label.txt')

test_source_ids, test_reply_ids = get_ids('test.data.txt')

In [4]:
import pandas as pd
import numpy as np

def create_df_and_dict(source_ids, source_labels):
    source_dict = {}
    for i in range(len(source_ids)):
        source_dict[source_ids[i]] = source_labels[i]

    df = pd.DataFrame(source_labels,source_ids)
    df.set_axis(['class'], axis='columns', inplace=True)
    return df, source_dict
    
train_df, train_dict = create_df_and_dict(train_source_ids, train_source_labels)
dev_df, dev_dict = create_df_and_dict(dev_source_ids, dev_source_labels)
train_df

,class
1250219300389974016,nonrumour
554886875303780352,rumour
1237901309011021825,nonrumour
524958128392376320,nonrumour
1239295488677085185,nonrumour
...,...
671181758692507648,rumour
672513234419638273,rumour
553508098825261056,nonrumour
1249479605582327808,nonrumour


In [5]:
api.lookup_statuses(['213321', '23231'],tweet_mode = "extended", map=True)[1].full_text

'RayreadRay - see what you mean about needing it tucked in - good grief...'

In [6]:
def get_tweets(source_ids):
    source_tweets = {}

    source_ids_split = [source_ids[x:x+100] for x in range(0, len(source_ids), 100)]

    for i in range(len(source_ids_split)):
        for tweet in api.lookup_statuses(source_ids_split[i],tweet_mode = "extended"):
            source_tweets[tweet.id_str] = tweet
        
    return source_tweets

def get_test_tweets(source_ids):
    source_tweets = {}

    source_ids_split = [source_ids[x:x+100] for x in range(0, len(source_ids), 100)]

    for i in range(len(source_ids_split)):
        for index, tweet in enumerate(api.lookup_statuses(source_ids_split[i],tweet_mode = "extended", map=True)):
            # Tries to get extended tweet otherwise uses tweet object to create tweet data
            try:
                tweet.full_text
                source_tweets[tweet.id_str] = tweet

            except:
                file = open(TWEET_OBJECT_PATH + str(tweet.id) + '.json')
                tweet = json.load(file)
                source_tweets[tweet['id']] = tweet
 
    return source_tweets

train_source_tweets = get_tweets(train_source_ids) 
dev_source_tweets = get_tweets(dev_source_ids)
test_source_tweets = get_test_tweets(test_source_ids)      
train_source_tweets['1255794736759140352']

Status(_api=<tweepy.api.API object at 0x0000023B8D7D2EF0>, _json={'created_at': 'Thu Apr 30 09:42:51 +0000 2020', 'id': 1255794736759140352, 'id_str': '1255794736759140352', 'full_text': "hi everything!! :)\nok, hi everyone!\njadi, berikut merupakan sedikit informasi untuk mengetahui what is covid-19?\nseperti yang kita tahu, ini merupakan virus yang paling baru d temukan,dan saat ini kita tengah di masa pandemi.\ndon't underestimate it! lets read✨ https://t.co/0jMMqGcwZH", 'truncated': False, 'display_text_range': [0, 260], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 1255794689485176834, 'id_str': '1255794689485176834', 'indices': [261, 284], 'media_url': 'http://pbs.twimg.com/media/EW16ndmWAAIkRV6.jpg', 'media_url_https': 'https://pbs.twimg.com/media/EW16ndmWAAIkRV6.jpg', 'url': 'https://t.co/0jMMqGcwZH', 'display_url': 'pic.twitter.com/0jMMqGcwZH', 'expanded_url': 'https://twitter.com/alifarahmanii/status/1255794736759140352/photo/1'

In [7]:
# print(source_tweets['1255794736759140352'].text)
# print(source_tweets['1255794736759140352'].user.screen_name)
# print(source_tweets['1255794736759140352'].user.verified)
# print(source_tweets['1255794736759140352'].user.followers_count)
# print(source_tweets['1255794736759140352'].truncated)
num_trunc = 0
total = len(train_source_tweets.keys())
for obj in train_source_tweets.values():
    if obj.truncated: num_trunc += 1
print(num_trunc/total)

0.0


In [46]:
def create_df2(source_tweets):
    tweet_text = []
    tweet_user = []
    tweet_verified = []
    tweet_follows = []
    tweet_hashtag = []
    
    for tweet in source_tweets.values():
        if type(tweet) is dict:
            tweet_text.append(tweet['text'])
            tweet_user.append(tweet['user']['screen_name'])
            tweet_verified.append(tweet['user']['verified'])
            tweet_follows.append(tweet['user']['followers_count'])
            if tweet['entities']['hashtags'] == []:
                tweet_hashtag.append("")
            else:
                tweet_hashtag.append(tweet['entities']['hashtags'][0]['text'])
        else:
            tweet_text.append(tweet.full_text)
            
            tweet_user.append(tweet.user.screen_name)
            tweet_verified.append(tweet.user.verified)
            tweet_follows.append(tweet.user.followers_count)
            if tweet.entities['hashtags'] == []:
                tweet_hashtag.append("")
            else:
                tweet_hashtag.append(tweet.entities['hashtags'][0]['text'])
   

    df2 = pd.DataFrame({'user':tweet_user,
                        'verified':tweet_verified,
                        'followers':tweet_follows,
                        'text': tweet_text,
                        'hashtag': tweet_hashtag},index=list(source_tweets.keys()))
    #df2['text_trimmed'] = df2['text'].str[:50]
    #df2.drop_duplicates(subset=['text_trimmed'],keep='first',inplace=True)

    return df2

    
train_df2 = create_df2(train_source_tweets)
dev_df2 = create_df2(dev_source_tweets)
test_df2 = create_df2(test_source_tweets)
train_df2

,user,verified,followers,text,hashtag
1255794736759140352,choc0onut,False,42,"hi everything!! :)\nok, hi everyone!\njadi, be...",
553586897168392192,France24_en,True,335283,#BREAKING : Both Charlie Hebdo suspects killed...,BREAKING
1238978947863523330,andoconmando39,True,3951,What are the symptoms of COVID-19? Please use...,COVID19
1251809538199732227,DrZweliMkhize,True,596109,How can I grocery shop safely in the time of #...,COVID_19
1235014618910728196,mailtribune,False,9200,#Editorial: What is a #coronavirus? How danger...,Editorial
...,...,...,...,...,...
1246063360740909057,DTalkscool,False,4277,@NCDCgov @NOA_Nigeria @FMICNigeria @Fmohnigeri...,
1239238887194554369,Chaganty,False,173,@BTelugabbayi https://t.co/WylIVfuTgU ...,
1242063054335967232,karunanundy,True,183662,"From WHO:\n ""Can the virus that causes COVID-1...",
1234884634795577346,teawithdev,False,8383,"Is there a vaccine, drug or treatment for COVI...",


In [34]:
def create_final_df(df1, df2):
    df_source = df1.join(df2)
    cols = df_source.columns.tolist()
    df_source = df_source[cols[1:8]+cols[0:1]]
    return df_source

train_final_df = create_final_df(train_df, train_df2)
dev_final_df = create_final_df(dev_df, dev_df2)

train_final_df

,user,verified,followers,text,retweets,favorites,hashtag,class
1218255692831903744,WHO,True,11151650.0,Q: Is there a treatment for a novel #coronavir...,202.0,265.0,coronavirus,nonrumour
1218269428166602753,WHO,True,11151650.0,Q: What can I do to protect myself from #coron...,2010.0,2058.0,coronavirus,nonrumour
1219979825848442880,WHO,True,11151650.0,@DrTedros @WHOWPRO @WHOSEARO @pahowho @WHOEMRO...,20.0,43.0,coronavirus,nonrumour
1219981282928668672,WHO,True,11151650.0,@DrTedros @WHOWPRO @WHOSEARO @pahowho @WHOEMRO...,21.0,49.0,coronavirus,nonrumour
1219982106757148673,WHO,True,11151650.0,@DrTedros @WHOWPRO @WHOSEARO @pahowho @WHOEMRO...,19.0,44.0,coronavirus,nonrumour
...,...,...,...,...,...,...,...,...
676718762830221312,Complex,True,2573410.0,Texas plumber sues car dealership after his tr...,170.0,257.0,,rumour
676870737932742656,ABC,True,17456732.0,Plumber suing car dealership for $1M after tru...,157.0,99.0,,rumour
677099574855639044,ABC,True,17456732.0,Plumber suing car dealership for $1M after tru...,104.0,80.0,,rumour
681767380305985536,mashable,True,9428562.0,"Sorry, Mark Zuckerberg is not giving $4.5 bill...",96.0,94.0,,rumour


In [12]:
def reply_create_lists(reply_ids_dict, source_dict, test):
    reply_source_ids = []
    reply_ids = []
    reply_source_class = []

    for source_id in reply_ids_dict.keys():
        for reply_id in reply_ids_dict[source_id]:
            reply_source_ids.append(source_id)
            reply_ids.append(reply_id)

    if not test:
        for source_id in reply_source_ids:
            reply_source_class.append(source_dict[source_id])
        assert(len(reply_source_ids)==len(reply_ids))
        return reply_source_ids, reply_ids, reply_source_class
    else:
        assert(len(reply_source_ids)==len(reply_ids))
        return reply_source_ids, reply_ids
        
    
    

train_reply_source_ids, train_reply_ids2, train_reply_source_class = reply_create_lists(train_reply_ids, train_dict, False)
dev_reply_source_ids, dev_reply_ids2, dev_reply_source_class = reply_create_lists(dev_reply_ids, dev_dict, False)
test_reply_source_ids, test_reply_ids2 = reply_create_lists(test_reply_ids, None, True)

train_reply_source_ids

['1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '1250219300389974016',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '554886875303780352',
 '55488687530378035

In [13]:
"""
    reply_tweets = {}

    reply_ids_split = [reply_ids[x:x+100] for x in range(0, len(reply_ids), 100)]

    for i in range(len(reply_ids_split)):
        for tweet in api.lookup_statuses(reply_ids_split[i],tweet_mode = "extended"):
            reply_tweets[tweet.id_str] = tweet
"""

train_reply_tweets = get_tweets(train_reply_ids2)
dev_reply_tweets = get_tweets(dev_reply_ids2)
test_reply_tweets = get_test_tweets(test_reply_ids2)
        
train_reply_tweets['554919484473880576']


Status(_api=<tweepy.api.API object at 0x0000023B8D7D2EF0>, _json={'created_at': 'Tue Jan 13 08:34:49 +0000 2015', 'id': 554919484473880576, 'id_str': '554919484473880576', 'full_text': '@Telegraph we will be very satisfied if #NawazSharif resigns. Here they hide the reports which blames them.', 'truncated': False, 'display_text_range': [0, 107], 'entities': {'hashtags': [{'text': 'NawazSharif', 'indices': [40, 52]}], 'symbols': [], 'user_mentions': [{'screen_name': 'Telegraph', 'name': 'The Telegraph', 'id': 16343974, 'id_str': '16343974', 'indices': [0, 10]}], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': 554886875303780352, 'in_reply_to_status_id_str': '554886875303780352', 'in_reply_to_user_id': 16343974, 'in_reply_to_user_id_str': '16343974', 'in_reply_to_screen_name': 'Telegraph', 'user': {'id': 518268340, 'id_str': '518268340', 'name': 'RK', 'screen_name': 'Immorality101', 'location': 'Here an

In [14]:
num_trunc = 0
total = len(train_reply_tweets.keys())
for obj in train_reply_tweets.values():
    if obj.truncated: num_trunc += 1
print(num_trunc/total)

0.0


In [16]:
def create_reply_df(reply_ids, reply_tweets, reply_source_ids, reply_source_class, test):
    tweet_text = []
    tweet_user = []
    tweet_verified = []
    tweet_follows = []
    tweet_retweets = []
    tweet_favorites = []
    tweet_hashtag = []

    for id in reply_ids:
        try:
            tweet = reply_tweets[id]
        except:
            pass
        if type(tweet) is dict:
            tweet_text.append(tweet['text'])
            tweet_user.append(tweet['user']['screen_name'])
            tweet_verified.append(tweet['user']['verified'])
            tweet_follows.append(tweet['user']['followers_count'])
            tweet_retweets.append(tweet['favorite_count'])
            tweet_retweets.append(tweet['retweet_count'])
            if tweet['entities']['hashtags'] == []:
                tweet_hashtag.append("")
            else:
                tweet_hashtag.append(tweet['entities']['hashtags'][0]['text'])
        else:
            tweet_text.append(tweet.full_text)
            tweet_user.append(tweet.user.screen_name)
            tweet_verified.append(tweet.user.verified)
            tweet_follows.append(tweet.user.followers_count)
            tweet_retweets.append(tweet.retweet_count)
            tweet_favorites.append(tweet.favorite_count)
            if tweet.entities['hashtags'] == []:
                tweet_hashtag.append("")
            else:
                tweet_hashtag.append(tweet.entities['hashtags'][0]['text'])

    if not test:
        df_replies = pd.DataFrame({'user':tweet_user,
                            'verified':tweet_verified,
                            'followers':tweet_follows,
                            'text':tweet_text,
                            'retweets':tweet_retweets,
                            'favorites':tweet_favorites,                                   
                            'hashtag':tweet_hashtag,
                            'source id':reply_source_ids,
                            'source class':reply_source_class,
                            },
                                index=reply_ids)
    else:
        df_replies = pd.DataFrame({'user':tweet_user,
                            'verified':tweet_verified,
                            'followers':tweet_follows,
                            'text':tweet_text,
                            'retweets':tweet_retweets,
                            'favorites':tweet_favorites, 
                            'hashtag':tweet_hashtag,
                            'source id':reply_source_ids,
                            },
                                index=reply_ids)
    
    return df_replies

In [35]:
train_df_replies = create_reply_df(train_reply_ids2, train_reply_tweets, train_reply_source_ids, train_reply_source_class, False)
dev_df_replies = create_reply_df(dev_reply_ids2, dev_reply_tweets, dev_reply_source_ids, dev_reply_source_class, False)
test_df_replies = create_reply_df(test_reply_ids2, test_reply_tweets, test_reply_source_ids, None, True)
train_df_replies    

,user,verified,followers,text,retweets,favorites,hashtag,source id,source class
1250219116993974272,ucoptempe,False,411,4. Can eating garlic help prevent infection wi...,0,0,COVID19Malaysia,1250219300389974016,nonrumour
1250219437027766273,ucoptempe,False,411,6. Do vaccines against pneumonia protect you a...,0,0,,1250219300389974016,nonrumour
1250219620939657216,ucoptempe,False,411,7. Can spraying alcohol or chlorine all over y...,0,0,Chamber,1250219300389974016,nonrumour
1250219777185873922,ucoptempe,False,411,8. How effective are thermal scanners in detec...,0,0,,1250219300389974016,nonrumour
1250219894429208577,ucoptempe,False,411,9. Can an ultraviolet disinfection lamp kill t...,0,0,,1250219300389974016,nonrumour
...,...,...,...,...,...,...,...,...,...
1241078443179155457,lynneSimpkin,False,561,@rosierawle All good. Just been working out th...,0,1,,1241082793737818113,nonrumour
1241085785060847617,lynneSimpkin,False,561,@rosierawle Best timetable I've heard about! A...,0,1,,1241082793737818113,nonrumour
1241086391267872770,rosierawle,False,3099,@lynneSimpkin And this exactly why you need to...,0,1,,1241082793737818113,nonrumour
1241086505566732289,lynneSimpkin,False,561,@rosierawle I might have sent that twice by ac...,0,1,,1241082793737818113,nonrumour


In [36]:
train_final_df

,user,verified,followers,text,retweets,favorites,hashtag,class
1218255692831903744,WHO,True,11151650.0,Q: Is there a treatment for a novel #coronavir...,202.0,265.0,coronavirus,nonrumour
1218269428166602753,WHO,True,11151650.0,Q: What can I do to protect myself from #coron...,2010.0,2058.0,coronavirus,nonrumour
1219979825848442880,WHO,True,11151650.0,@DrTedros @WHOWPRO @WHOSEARO @pahowho @WHOEMRO...,20.0,43.0,coronavirus,nonrumour
1219981282928668672,WHO,True,11151650.0,@DrTedros @WHOWPRO @WHOSEARO @pahowho @WHOEMRO...,21.0,49.0,coronavirus,nonrumour
1219982106757148673,WHO,True,11151650.0,@DrTedros @WHOWPRO @WHOSEARO @pahowho @WHOEMRO...,19.0,44.0,coronavirus,nonrumour
...,...,...,...,...,...,...,...,...
676718762830221312,Complex,True,2573410.0,Texas plumber sues car dealership after his tr...,170.0,257.0,,rumour
676870737932742656,ABC,True,17456732.0,Plumber suing car dealership for $1M after tru...,157.0,99.0,,rumour
677099574855639044,ABC,True,17456732.0,Plumber suing car dealership for $1M after tru...,104.0,80.0,,rumour
681767380305985536,mashable,True,9428562.0,"Sorry, Mark Zuckerberg is not giving $4.5 bill...",96.0,94.0,,rumour


In [48]:
train_final_df.index.name = 'id'
train_df_replies.index.name = 'id'
train_final_df.dropna(subset=['text'],inplace=True)
train_df_replies.dropna(subset=['text'],inplace=True)
train_final_df.to_csv('train_sources.csv', header=True)
train_df_replies.to_csv('train_replies.csv', header=True)

In [49]:
dev_final_df.index.name = 'id'
dev_df_replies.index.name = 'id'
dev_final_df.dropna(subset=['text'],inplace=True)
dev_df_replies.dropna(subset=['text'],inplace=True)
dev_final_df.to_csv('dev_sources.csv', header=True)
dev_df_replies.to_csv('dev_replies.csv', header=True)

In [50]:
test_df2.index.name = 'id'
test_df_replies.index.name = 'id'
test_df2.dropna(subset=['text'],inplace=True)
test_df_replies.dropna(subset=['text'],inplace=True)
test_df2.to_csv('test_sources.csv', header=True)
test_df_replies.to_csv('test_replies.csv', header=True)

In [51]:
a = pd.read_csv('train_sources.csv', index_col=0)
b = pd.read_csv('train_replies.csv', index_col=0)

In [53]:
a

,user,verified,followers,text,retweets,favorites,hashtag,class
id,,,,,,,,
1218255692831903744,WHO,True,11151650.0,Q: Is there a treatment for a novel #coronavir...,202.0,265.0,coronavirus,nonrumour
1218269428166602753,WHO,True,11151650.0,Q: What can I do to protect myself from #coron...,2010.0,2058.0,coronavirus,nonrumour
1219979825848442880,WHO,True,11151650.0,@DrTedros @WHOWPRO @WHOSEARO @pahowho @WHOEMRO...,20.0,43.0,coronavirus,nonrumour
1219981282928668672,WHO,True,11151650.0,@DrTedros @WHOWPRO @WHOSEARO @pahowho @WHOEMRO...,21.0,49.0,coronavirus,nonrumour
1219982106757148673,WHO,True,11151650.0,@DrTedros @WHOWPRO @WHOSEARO @pahowho @WHOEMRO...,19.0,44.0,coronavirus,nonrumour
...,...,...,...,...,...,...,...,...
676367888543031296,nycjim,True,211884.0,North Carolina town rejects solar panels for s...,329.0,170.0,NaN,nonrumour
676718762830221312,Complex,True,2573410.0,Texas plumber sues car dealership after his tr...,170.0,257.0,NaN,rumour
676870737932742656,ABC,True,17456732.0,Plumber suing car dealership for $1M after tru...,157.0,99.0,NaN,rumour


In [54]:
b

,user,verified,followers,text,retweets,favorites,hashtag,source id,source class
id,,,,,,,,,
1250219116993974272,ucoptempe,False,411,4. Can eating garlic help prevent infection wi...,0,0,COVID19Malaysia,1250219300389974016,nonrumour
1250219437027766273,ucoptempe,False,411,6. Do vaccines against pneumonia protect you a...,0,0,NaN,1250219300389974016,nonrumour
1250219620939657216,ucoptempe,False,411,7. Can spraying alcohol or chlorine all over y...,0,0,Chamber,1250219300389974016,nonrumour
1250219777185873922,ucoptempe,False,411,8. How effective are thermal scanners in detec...,0,0,NaN,1250219300389974016,nonrumour
1250219894429208577,ucoptempe,False,411,9. Can an ultraviolet disinfection lamp kill t...,0,0,NaN,1250219300389974016,nonrumour
...,...,...,...,...,...,...,...,...,...
1241078443179155457,lynneSimpkin,False,561,@rosierawle All good. Just been working out th...,0,1,NaN,1241082793737818113,nonrumour
1241085785060847617,lynneSimpkin,False,561,@rosierawle Best timetable I've heard about! A...,0,1,NaN,1241082793737818113,nonrumour
1241086391267872770,rosierawle,False,3099,@lynneSimpkin And this exactly why you need to...,0,1,NaN,1241082793737818113,nonrumour
